In [31]:
import pathlib
import pandas as pd
import xml.etree.ElementTree as et

# define the path
currentDirectory = pathlib.Path('XML')

# define the pattern
currentPattern = "*.xml"
all_xml_files = []
for currentFile in currentDirectory.glob(currentPattern):
    all_xml_files.append(currentFile.parts[-1])
    
print(all_xml_files)

['Brother_Bear.xml', 'American_Pie.xml', 'Andaz_Apna_Apna.xml', 'The_Fault_in_Our_Stars.xml', 'Shakespeare_in_Love.xml', 'Night_at_the_Museum__Secret_of_the_Tomb.xml', 'Live_Nude_Girls.xml', 'The_Babadook.xml', 'The_Hobbit__The_Desolation_of_Smaug.xml', 'Boogie_Nights.xml', 'V_for_Vendetta.xml', 'The_Theory_of_Everything.xml', 'Robin_Hood.xml', 'The_Phantom_Tollbooth.xml', 'Ode_to_My_Father.xml', 'Mulan.xml', 'Bhoothnath_Returns.xml', 'The_Reluctant_Dragon.xml', 'The_Dark_Knight.xml', 'Elsa___Fred.xml', 'Lord_of_the_Flies.xml', 'My_Week_with_Marilyn.xml', 'Mean_Girls.xml', 'Invincible.xml', 'Hotel_Transylvania.xml', 'Khumba.xml', 'The_Bourne_Legacy.xml', 'Bros_Before_Hos.xml', 'Hook.xml', 'Black_Swan.xml', 'Still_Alice.xml', 'Exodus__Gods_and_Kings.xml', 'Chef.xml', 'The_Hunger_Games__Catching_Fire.xml', 'Into_the_Wild.xml', 'A_Single_Man.xml', 'The_Blair_Witch_Project.xml', 'Looney_Tunes__Back_in_Action.xml', 'The_Reader.xml', 'Total_Recall.xml', 'The_Imitation_Game.xml', 'Maine_Pyar_

In [32]:
def parse_XML(xml_file, name, df_cols):
            global out_df

            xtree = et.parse(xml_file)
            xroot = xtree.getroot()
            fin = []
            res = []
            res.append(name.split(".")[0])
            
            for node in xroot:
                for i in node_cols:
                    res.append(node.attrib.get(i))

            fin.append(res)
            print(fin)
            out_df = pd.DataFrame(data=fin, columns =meta_cols)
            return out_df
        
def parse_XML1(xml_file, name, df_cols, df):
            global out_df
            xtree = et.parse(xml_file)
            xroot = xtree.getroot()
            fin = []
            res = []
            if name.split(".")[0] == "Mr":
                res.append("Mr._Go")
            else:
                res.append(name.split(".")[0])
            for node in xroot:
                for i in node_cols:
                    res.append(node.attrib.get(i))

            fin.append(res)
            df1 = pd.DataFrame(data=fin, columns =meta_cols)
            
            frames = [df, df1]
            
            out_df = pd.concat(frames)
            
            return out_df

In [33]:
meta_cols = ["filename","language", "year", "genre", "country",
            "runtime", "rated"]

node_cols = ["language", "year", "genre", "country",
            "runtime", "rated"]

user_rating_cols = ["metascore", "imdbRating", "tomatoRating", "tomatoUserRating"]
user_and_meta_cols = ["language", "year", "genre", "country",
            "runtime", "rated", "metascore", "imdbRating", "tomatoRating", "tomatoUserRating"]
user = True
meta = True

In [34]:
if user == True:
    #meta_cols=["filename","metascore", "imdbRating", "tomatoMeter", "tomatoUserMeter"]
    meta_cols=["filename","language", "year", "genre", "country",
            "runtime", "rated"]
    node_cols=user_rating_cols
if user == True and meta == True:
    meta_cols=["filename","language", "year", "genre", "country",
            "runtime", "rated", "metascore", "imdbRating", "tomatoRating", "tomatoUserRating"]
    node_cols=user_and_meta_cols
c = 1
for i in all_xml_files:
    if c==1:
        parse_XML("XML/" + i, i,  meta_cols)
        c=43
    else:
        parse_XML1("XML/"+i, i, meta_cols, out_df)

[['Brother_Bear', 'English, Inuktitut, Croatian, Serbian', '2003', 'Animation, Adventure, Family', 'USA', '85 min', 'G', '48', '6.7', '5.5', '3.3']]


In [35]:
df_ground = pd.read_excel("dev_set_groundtruth_and_trailers.xls",index_col=False)

In [36]:
a = df_ground["filename"].sort_values()
b = out_df["filename"].sort_values()

for i in a.values:
    if i in b.values:
        continue
    else:
        print(i)

In [37]:
df_join = pd.merge(df_ground, out_df, on="filename", how="inner")
#df_join = df_join.drop(["movie", "filename", "trailer"], axis=1)
df_join

,movie,filename,goodforairplane,trailer,language,year,genre,country,runtime,rated,metascore,imdbRating,tomatoRating,tomatoUserRating
0,Seventh Son,Seventh_Son,1,http://www.imdb.com/video/imdb/vi1803464473/?r...,English,2014,"Action, Adventure, Fantasy","USA, UK, Canada, China",102 min,PG-13,30,5.5,3.8,2.9
1,Welcome to Me,Welcome_to_Me,0,http://www.imdb.com/video/imdb/vi226471449/?re...,English,2014,"Comedy, Drama",USA,105 min,R,67,6.2,6.4,3.4
2,The Judge,The_Judge,0,http://www.imdb.com/video/imdb/vi1286123289/?r...,English,2014,Drama,USA,141 min,R,48,7.5,5.6,3.8
3,Transformers Age of Extinction,Transformers__Age_of_Extinction,0,http://www.imdb.com/video/imdb/vi3138759961/?r...,English,2014,"Action, Adventure, Sci-Fi","USA, China",165 min,PG-13,32,5.8,3.9,3.3
4,The Normal Heart,The_Normal_Heart,1,http://www.imdb.com/video/imdb/vi2749541657/?r...,English,2014,Drama,USA,132 min,TV-MA,N/A,8.0,7.7,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Live Nude Girls,Live_Nude_Girls,0,https://www.youtube.com/watch?v=RrjF-FSFrfo,English,1995,Comedy,USA,95 min,R,N/A,5.4,4.4,3.2
91,Pulp Fiction,Pulp_Fiction,0,https://www.youtube.com/watch?v=s7EdQ4FqbhY,"English, Spanish, French",1994,"Crime, Drama",USA,154 min,R,94,8.9,9.0,4.2
92,Perez,Perez,0,https://www.youtube.com/watch?v=HsXRKgzs1Kk,Italian,2014,"Crime, Drama, Thriller",Italy,94 min,N/A,N/A,6.5,N/A,4.2
93,A Goofy Movie,A_Goofy_Movie,0,https://www.youtube.com/watch?v=Ou94SEYa9jI,English,1995,"Animation, Adventure, Comedy",USA,78 min,G,N/A,6.8,5.6,3.1


In [38]:
import numpy as np
if user == False or meta == True:
    count=0
    for i in df_join["genre"].values:
        df_join["genre"].values[count] = i.replace(" ", "")
        count+=1

    count=0
    for i in df_join["language"].values:
        df_join["language"].values[count] = i.replace(" ", "")
        count+=1


    count=0
    for i in df_join["country"].values:
        df_join["country"].values[count] = i.replace(" ", "")
        count+=1
        
    genre = df_join.genre.str.split(",", expand=True).stack()
    language = df_join.language.str.split(",", expand=True).stack()
    country = df_join.country.str.split(",", expand=True).stack()
    
    df_join = pd.concat([df_join, 
                pd.get_dummies(genre, prefix="g").groupby(level=0).sum()],axis=1) \
                .drop(["genre"],axis=1)

    df_join = pd.concat([df_join, 
                    pd.get_dummies(language, prefix="l").groupby(level=0).sum()],axis=1) \
                    .drop(["language"],axis=1)

    df_join = pd.concat([df_join, 
                    pd.get_dummies(country, prefix="c").groupby(level=0).sum()],axis=1) \
                    .drop(["country"],axis=1)

In [39]:
df_join

,movie,filename,goodforairplane,trailer,year,runtime,rated,metascore,imdbRating,tomatoRating,...,c_Ireland,c_Italy,c_Japan,c_Netherlands,c_NewZealand,c_SouthAfrica,c_SouthKorea,c_Spain,c_UK,c_USA
0,Seventh Son,Seventh_Son,1,http://www.imdb.com/video/imdb/vi1803464473/?r...,2014,102 min,PG-13,30,5.5,3.8,...,0,0,0,0,0,0,0,0,1,1
1,Welcome to Me,Welcome_to_Me,0,http://www.imdb.com/video/imdb/vi226471449/?re...,2014,105 min,R,67,6.2,6.4,...,0,0,0,0,0,0,0,0,0,1
2,The Judge,The_Judge,0,http://www.imdb.com/video/imdb/vi1286123289/?r...,2014,141 min,R,48,7.5,5.6,...,0,0,0,0,0,0,0,0,0,1
3,Transformers Age of Extinction,Transformers__Age_of_Extinction,0,http://www.imdb.com/video/imdb/vi3138759961/?r...,2014,165 min,PG-13,32,5.8,3.9,...,0,0,0,0,0,0,0,0,0,1
4,The Normal Heart,The_Normal_Heart,1,http://www.imdb.com/video/imdb/vi2749541657/?r...,2014,132 min,TV-MA,N/A,8.0,7.7,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Live Nude Girls,Live_Nude_Girls,0,https://www.youtube.com/watch?v=RrjF-FSFrfo,1995,95 min,R,N/A,5.4,4.4,...,0,0,0,0,0,0,0,0,0,1
91,Pulp Fiction,Pulp_Fiction,0,https://www.youtube.com/watch?v=s7EdQ4FqbhY,1994,154 min,R,94,8.9,9.0,...,0,0,0,0,0,0,0,0,0,1
92,Perez,Perez,0,https://www.youtube.com/watch?v=HsXRKgzs1Kk,2014,94 min,N/A,N/A,6.5,N/A,...,0,1,0,0,0,0,0,0,0,0
93,A Goofy Movie,A_Goofy_Movie,0,https://www.youtube.com/watch?v=Ou94SEYa9jI,1995,78 min,G,N/A,6.8,5.6,...,0,0,0,0,0,0,0,0,0,1


In [40]:
if user == True:
    df_join.replace("N/A", np.nan, inplace=True)


In [45]:
df_join.dtypes
df_join = df_join.drop(["movie", "filename","trailer"], axis=1)

In [43]:
df_join.columns
df_join = df_join.astype("category")

movie              category
filename           category
goodforairplane    category
trailer            category
year               category
                     ...   
c_SouthAfrica      category
c_SouthKorea       category
c_Spain            category
c_UK               category
c_USA              category
Length: 76, dtype: object

In [46]:
df_join.to_csv("meta_plus_ratings.csv",index=False)

=== Detailed Accuracy By Class === user ratings

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,233    0,346    0,357      0,233    0,282      -0,124   0,413     0,400     '(-inf-0.5]'
                 0,654    0,767    0,507      0,654    0,571      -0,124   0,416     0,489     '(0.5-inf)'
Weighted Avg.    0,463    0,577    0,439      0,463    0,440      -0,124   0,415     0,449     



=== Detailed Accuracy By Class === meta+rating

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,116    0,077    0,556      0,116    0,192      0,067    0,522     0,489     '(-inf-0.5]'
                 0,923    0,884    0,558      0,923    0,696      0,067    0,522     0,558     '(0.5-inf)'
Weighted Avg.    0,558    0,519    0,557      0,558    0,468      0,067    0,522     0,526     


In [107]:
## VISUAL 
# define the path
currentDirectory = pathlib.Path('D:/TU/ED/EX2/CoE_dataset/Dev_Set/vis_descriptors')

# define the pattern
currentPattern = "*.csv"
all_visual_files = []
for currentFile in currentDirectory.glob(currentPattern):
    all_visual_files.append(currentFile.parts[-1])
    
print(all_visual_files)

['American_Gangster.csv', 'American_Pie.csv', 'Andaz_Apna_Apna.csv', 'Anna_Karenina.csv', 'A_Fish_Called_Wanda.csv', 'A_Goofy_Movie.csv', 'A_Million_Ways_to_Die_in_the_West.csv', 'A_Single_Man.csv', 'Babar__The_Movie.csv', 'Bhoothnath_Returns.csv', 'Big_Hero_6.csv', 'Black_Swan.csv', 'Bobby_Jasoos.csv', 'Boogie_Nights.csv', 'Bronson.csv', 'Bros_Before_Hos.csv', 'Brother_Bear.csv', 'Casino_Royale.csv', 'Chef.csv', 'Dirty_Dancing.csv', 'Donnie_Darko.csv', 'Drive.csv', 'Elsa___Fred.csv', 'Excuse_My_French.csv', 'Exodus__Gods_and_Kings.csv', 'Full_Metal_Jacket.csv', 'Grand_Piano.csv', 'Homies.csv', 'Hook.csv', 'Hotel_Transylvania.csv', 'Inherent_Vice.csv', 'Interstellar.csv', 'Into_the_Wild.csv', 'Into_the_Woods.csv', 'Invincible.csv', 'Jack_Reacher.csv', 'Jerry_Maguire.csv', 'John_Wick.csv', 'Khoobsurat.csv', 'Khumba.csv', "Let's_Be_Cops.csv", 'Le_Samourai.csv', 'Live_Nude_Girls.csv', 'Looney_Tunes__Back_in_Action.csv', 'Lord_of_the_Flies.csv', 'Maine_Pyar_Kiya.csv', 'Mean_Girls.csv', 'Mo

In [81]:
df = pd.read_csv("vis_descriptors/" + all_visual_files[0], header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,816,817,818,819,820,821,822,823,824,825
0,0.23164,0.28629,0.30068,0.28118,0.24489,0.27849,0.29076,0.2938,0.16104,0.16685,...,232.17,109.62,17.237,15.543,13.671,15.231,53559,8637.4,18597,8679
1,0.23164,0.28629,0.30068,0.28118,0.24489,0.27849,0.29076,0.2938,0.16104,0.16685,...,232.17,109.62,17.237,15.543,13.671,15.231,53559,8637.4,18597,8679


In [130]:
c = 1

for i in all_visual_files:
    if c==1:
        df_vis = pd.read_csv("vis_descriptors/" + i, header=None)
        all_features = []
        for index, row in df_vis.iterrows():
            for j in row.values:
                all_features.append(j)
        df_vis_concat = pd.DataFrame([all_features], columns=list(range(0, 1652)))
        name = i[:-4]
        df_vis_concat.insert(0, "filename", name)
        first_row = df_vis_concat.values.tolist()
        c=43
    else:
        df_vis = pd.read_csv("vis_descriptors/" + i, header=None)
        all_features = []
        for index, row in df_vis.iterrows():
            for j in row.values:
                all_features.append(j)
        df_temp = pd.DataFrame([all_features], columns=list(range(0, 1652)))
        name = i[:-4]
        df_temp.insert(0, "filename", name)
        first_row.append(df_temp.values.tolist()[0])
df_vis_fin = pd.DataFrame(first_row)
df_vis_fin

,0,1,2,3,4,5,6,7,8,9,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652
0,American_Gangster,0.231640,0.286290,0.300680,0.281180,0.244890,0.278490,0.290760,0.293800,0.161040,...,232.170,109.6200,17.237000,15.543000,13.671000,15.231000,53559.0,8637.40,18597.00,8679.00
1,American_Pie,0.237360,0.250190,0.255130,0.260250,0.193180,0.208400,0.295470,0.363410,0.241420,...,389.280,112.0300,1.481500,2.082400,1.675600,2.140800,2639.4,1377.00,4321.30,1299.10
2,Andaz_Apna_Apna,0.000000,0.294160,0.290070,0.011351,0.100930,0.355760,0.364370,0.236320,0.100780,...,57486.000,20076.0000,9.715800,10.036000,7.630700,11.156000,66445.0,27016.00,60427.00,20459.00
3,Anna_Karenina,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,230400.000,119950.0000,0.000001,0.002466,0.000004,0.002466,729320.0,119950.00,230400.00,119950.00
4,A_Fish_Called_Wanda,0.430310,0.381010,0.340820,0.316420,0.414650,0.385990,0.329380,0.312120,0.350670,...,765.820,362.8300,8.592300,9.142700,8.410100,8.792400,1483.3,417.21,892.59,435.28
5,A_Goofy_Movie,0.002031,0.000000,0.000000,0.073020,0.027533,0.005346,0.006015,0.118240,0.026991,...,33542.000,20278.0000,0.972010,1.365400,1.803200,1.463400,168740.0,20896.00,34434.00,20967.00
6,A_Million_Ways_to_Die_in_the_West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,230400.000,119950.0000,0.000001,0.002466,0.000004,0.002466,729320.0,119950.00,230400.00,119950.00
7,A_Single_Man,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,21667.000,9550.7000,5.943300,6.021500,4.383900,5.428800,32242.0,10313.00,22308.00,9850.00
8,Babar__The_Movie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,129600.000,55609.0000,0.000004,0.003916,0.000008,0.003916,230400.0,55609.00,129600.00,55609.00
9,Bhoothnath_Returns,0.229340,0.136600,0.156360,0.361430,0.220950,0.237900,0.268820,0.360830,0.146440,...,7462.200,3027.9000,9.214400,10.289000,9.330100,9.628400,38404.0,10548.00,24415.00,10098.00


In [143]:

z = list(range(0,1652))
z.insert(0, "filename")
colnames = z
df_vis_fin.columns = colnames
df_vis_fin

,filename,0,1,2,3,4,5,6,7,8,...,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651
0,American_Gangster,0.231640,0.286290,0.300680,0.281180,0.244890,0.278490,0.290760,0.293800,0.161040,...,232.170,109.6200,17.237000,15.543000,13.671000,15.231000,53559.0,8637.40,18597.00,8679.00
1,American_Pie,0.237360,0.250190,0.255130,0.260250,0.193180,0.208400,0.295470,0.363410,0.241420,...,389.280,112.0300,1.481500,2.082400,1.675600,2.140800,2639.4,1377.00,4321.30,1299.10
2,Andaz_Apna_Apna,0.000000,0.294160,0.290070,0.011351,0.100930,0.355760,0.364370,0.236320,0.100780,...,57486.000,20076.0000,9.715800,10.036000,7.630700,11.156000,66445.0,27016.00,60427.00,20459.00
3,Anna_Karenina,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,230400.000,119950.0000,0.000001,0.002466,0.000004,0.002466,729320.0,119950.00,230400.00,119950.00
4,A_Fish_Called_Wanda,0.430310,0.381010,0.340820,0.316420,0.414650,0.385990,0.329380,0.312120,0.350670,...,765.820,362.8300,8.592300,9.142700,8.410100,8.792400,1483.3,417.21,892.59,435.28
5,A_Goofy_Movie,0.002031,0.000000,0.000000,0.073020,0.027533,0.005346,0.006015,0.118240,0.026991,...,33542.000,20278.0000,0.972010,1.365400,1.803200,1.463400,168740.0,20896.00,34434.00,20967.00
6,A_Million_Ways_to_Die_in_the_West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,230400.000,119950.0000,0.000001,0.002466,0.000004,0.002466,729320.0,119950.00,230400.00,119950.00
7,A_Single_Man,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,21667.000,9550.7000,5.943300,6.021500,4.383900,5.428800,32242.0,10313.00,22308.00,9850.00
8,Babar__The_Movie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,129600.000,55609.0000,0.000004,0.003916,0.000008,0.003916,230400.0,55609.00,129600.00,55609.00
9,Bhoothnath_Returns,0.229340,0.136600,0.156360,0.361430,0.220950,0.237900,0.268820,0.360830,0.146440,...,7462.200,3027.9000,9.214400,10.289000,9.330100,9.628400,38404.0,10548.00,24415.00,10098.00


In [182]:
df_join1 = pd.merge(df_ground, df_vis_fin, on="filename", how="inner")
df_join1 = df_join1.drop(["movie", "trailer"], axis=1)
df_join1

df_join = pd.merge(df_join, df_join1, on="filename", how="inner")
df_join  = df_join.drop(["filename", "movie", "trailer"], axis=1)
df_join


,goodforairplane_x,year,runtime,rated,g_Action,g_Adventure,g_Animation,g_Biography,g_Comedy,g_Crime,...,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651
0,1,2014,102 min,PG-13,1,1,0,0,0,0,...,689.950,474.970,2.267600,2.588700,2.402200,3.216700,81373.0,21045.00,24225.0,13529.00
1,0,2014,105 min,R,0,0,0,0,1,0,...,397.260,168.230,20.426000,21.300000,18.608000,21.182000,83171.0,13714.00,32774.0,13780.00
2,0,2014,141 min,R,0,0,0,0,0,0,...,230400.000,119950.000,0.000001,0.002466,0.000004,0.002466,729320.0,119950.00,230400.0,119950.00
3,0,2014,165 min,PG-13,1,1,0,0,0,0,...,1120.600,669.560,15.086000,14.700000,14.859000,14.723000,211630.0,24019.00,48339.0,24090.00
4,1,2014,132 min,TV-MA,0,0,0,0,0,0,...,41786.000,19786.000,13.071000,11.296000,11.202000,11.306000,79962.0,20617.00,45216.0,20738.00
5,1,1970,90 min,G,0,1,1,0,0,0,...,1644.800,268.640,0.592040,0.893830,0.721410,0.879250,17662.0,1955.10,6105.2,2000.10
6,1,1994,160 min,PG,0,0,0,0,1,0,...,57486.000,20076.000,9.715800,10.036000,7.630700,11.156000,66445.0,27016.00,60427.0,20459.00
7,1,2012,91 min,PG,0,0,1,0,1,0,...,230400.000,119790.000,0.000001,0.002470,0.000004,0.002470,725900.0,119790.00,230400.0,119790.00
8,1,1999,136 min,R,1,0,0,0,0,0,...,141.520,89.064,1.672600,1.844900,1.220700,1.884300,22739.0,3850.10,6361.4,3976.70
9,1,2007,148 min,R,0,1,0,1,0,0,...,485.350,308.520,18.196000,15.608000,13.551000,15.352000,26102.0,3523.40,6113.7,3568.10


In [183]:
df_join.to_csv("meta+visual.csv",index=False)

=== Detailed Accuracy By Class === visual

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,163    0,154    0,467      0,163    0,241      0,012    0,485     0,445     '(-inf-0.5]'
                 0,846    0,837    0,550      0,846    0,667      0,012    0,485     0,543     '(0.5-inf)'
Weighted Avg.    0,537    0,528    0,512      0,537    0,474      0,012    0,485     0,499 


=== Detailed Accuracy By Class === visual+meta

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,233    0,231    0,455      0,233    0,308      0,002    0,482     0,442     '(-inf-0.5]'
                 0,769    0,767    0,548      0,769    0,640      0,002    0,482     0,536     '(0.5-inf)'
Weighted Avg.    0,526    0,525    0,506      0,526    0,490      0,002    0,482     0,493 